In [1]:
%matplotlib widget
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
import torch.nn.functional as F
import torch
import yaml
import lpips
import scipy.signal

import sys
base_path = Path(os.path.abspath('')).parent
print(base_path)
sys.path.append(str(base_path))
import torch
from dataLoader import dataset_dict

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)


/home/dronelab/normalrf


In [2]:
def rgb_ssim(img0, img1, max_val,
             filter_size=11,
             filter_sigma=1.5,
             k1=0.01,
             k2=0.03,
             return_map=False):
    # Modified from https://github.com/google/mipnerf/blob/16e73dfdb52044dcceb47cda5243a686391a6e0f/internal/math.py#L58
    assert len(img0.shape) == 3
    assert img0.shape[-1] == 3
    assert img0.shape == img1.shape

    # Construct a 1D Gaussian blur filter.
    hw = filter_size // 2
    shift = (2 * hw - filter_size + 1) / 2
    f_i = ((np.arange(filter_size) - hw + shift) / filter_sigma)**2
    filt = np.exp(-0.5 * f_i)
    filt /= np.sum(filt)

    # Blur in x and y (faster than the 2D convolution).
    def convolve2d(z, f):
        return scipy.signal.convolve2d(z, f, mode='valid')

    filt_fn = lambda z: np.stack([
        convolve2d(convolve2d(z[...,i], filt[:, None]), filt[None, :])
        for i in range(z.shape[-1])], -1)
    mu0 = filt_fn(img0)
    mu1 = filt_fn(img1)
    mu00 = mu0 * mu0
    mu11 = mu1 * mu1
    mu01 = mu0 * mu1
    sigma00 = filt_fn(img0**2) - mu00
    sigma11 = filt_fn(img1**2) - mu11
    sigma01 = filt_fn(img0 * img1) - mu01

    # Clip the variances and covariances to valid values.
    # Variance must be non-negative:
    sigma00 = np.maximum(0., sigma00)
    sigma11 = np.maximum(0., sigma11)
    sigma01 = np.sign(sigma01) * np.minimum(
        np.sqrt(sigma00 * sigma11), np.abs(sigma01))
    c1 = (k1 * max_val)**2
    c2 = (k2 * max_val)**2
    numer = (2 * mu01 + c1) * (2 * sigma01 + c2)
    denom = (mu00 + mu11 + c1) * (sigma00 + sigma11 + c2)
    ssim_map = numer / denom
    ssim = np.mean(ssim_map)
    return ssim_map if return_map else ssim


device = torch.device('cuda')
loss_fn_alex = lpips.LPIPS(net='alex').to(device)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


In [3]:
def get_stats(p):
    stats_files = ['stats_augnle.yaml', 'stats_augnl.yaml', 'stats_augn.yaml', 'stats_aug.yaml', 'stats.yaml']
    stat_file = [s for s in stats_files if (p / "imgs_test_all" / s).exists()]
    stat_file = stat_file[0] if len(stat_file) else None
    if stat_file is None:
        return None
    data_path = p / "imgs_test_all" / stat_file
    with data_path.open('r') as f:
        data = yaml.safe_load(f)
    return data


In [4]:

# log_dir = Path("../log") / "tensorf"
# log_dir = Path("../log") / "fixedmip128"
# log_dir = Path("../log") / "singlebounce_samen"
# exps = ["noprednorms_nl0_conserve_pb0", 'fixedmip128', 'tensorf', 'singlebounce_samen']
# exps = ['interpdiffuse', 'interpdiffuse_flipnorm']
exps = ['noupsample']

stats_files = ['stats_augn.yaml', 'stats_aug.yaml', 'stats.yaml']
datadir = "/optane/nerf_datasets"
for exp in exps:
    log_dir = Path("../log") / exp
    for p in log_dir.glob("*"):
        print(p)
        pano_path = p / "imgs_test_all" / "envmaps" / "pano.exr"
        mapped_pano_path = p / "imgs_test_all" / "envmaps" / "mapped_pano.png"
        new_data_path = p / "imgs_test_all" / "stats_augnl.yaml"

        old_data = get_stats(p)

        config = OmegaConf.load(p / 'config.yaml')
        dname = Path(config['dataset']['scenedir']).name

        white_bg = config.dataset.white_bg if hasattr(config.dataset, 'white_bg') else True
        dataset = dataset_dict[config.dataset.dataset_name]
        test_dataset = dataset(os.path.join(datadir, config.dataset.scenedir), split='test',
                            downsample=config.dataset.downsample_train, is_stack=True, white_bg=white_bg, is_testing=True)
        lpips_vs = []
        ssims = []
        psnrs = []
        for idx in range(len(test_dataset)):
            pred_im = torch.as_tensor(imageio.imread(p / "imgs_test_all" / f"{idx:03d}.png")).float() / 255
            gt_im = test_dataset.all_rgbs[idx]

            pred_t = pred_im.permute(2, 0, 1).unsqueeze(0).to(device)
            gt_t = gt_im.permute(2, 0, 1).unsqueeze(0).to(device)
            err = (pred_im - gt_im)**2
            psnr = -10 * torch.log(torch.mean(err)) / np.log(10.0)
            psnrs.append(psnr)

            with torch.no_grad():
                lpips_v = loss_fn_alex(pred_t, gt_t, normalize=True).cpu()
            ssims.append(rgb_ssim(pred_im, gt_im, 1))
            lpips_vs.append(float(lpips_v))

        with new_data_path.open('w') as f:
            old_data['l_alex'] = float(sum(lpips_vs) / len(lpips_vs)) if len(lpips_vs) > 0 else 0
            old_data['new_psnr'] = float(sum(psnrs) / len(psnrs)) if len(psnrs) > 0 else 0
            old_data['ssim'] = float(sum(ssims) / len(ssims)) if len(ssims) > 0 else 0
            yaml.dump(old_data, f)


/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


../log/noupsample/hotdog_v38_noupsample


Loading data test (200): 100%|██████████| 200/200 [00:05<00:00, 36.60it/s]
/tmp/ipykernel_4928/819918741.py:31: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pred_im = torch.as_tensor(imageio.imread(p / "imgs_test_all" / f"{idx:03d}.png")).float() / 255


../log/noupsample/ball_v38_noupsample


Loading data test (200): 100%|██████████| 200/200 [00:05<00:00, 35.37it/s]


...

Loading data test (200): 100%|██████████| 200/200 [00:05<00:00, 39.23it/s]


../log/noupsample/chair_v38_noupsample


Loading data test (200): 100%|██████████| 200/200 [00:05<00:00, 39.08it/s]


../log/noupsample/teapot_v38_noupsample


Loading data test (200): 100%|██████████| 200/200 [00:04<00:00, 44.63it/s]
